`serve run translator:translator_app `

In [15]:
import ray
import time
from time import perf_counter
import requests

In [16]:
@ray.remote
def query_backend(text="Hello world!"):
    response = requests.post("http://127.0.0.1:8000/", json=text)
    return response.text


In [17]:
def as_available(things):
 # Make the futures
 futures = list(map(lambda x: query_backend.remote(x), things))
 # print(ray.get(futures[0]))
 # While we still have pending futures
 while len(futures) > 0:
    ready_futures, rest_futures = ray.wait(futures)
    # print(f"Ready {len(ready_futures)} rest {len(rest_futures)}")
    # for id in ready_futures:
    #     print(f'completed value {id}, result {ray.get(id)}')
        # time.sleep(1) # Business logic goes here
    # We just need to wait on the ones that are not yet available
    futures = rest_futures


In [19]:
things = ["Hello world!", "How are you?", "The sun is shinning!"] * 100
t1_start = perf_counter() 
as_available(things)
t1_stop = perf_counter()
print(f"Elapsed time during the {len(things)} requests in seconds:",
                                        t1_stop-t1_start)

Elapsed time during the 300 requests in seconds: 43.327790417009965


In [11]:
from transformers import pipeline

class Translator:
    def __init__(self):
        # Load model
        self.model = pipeline("translation_en_to_de", model="t5-small")

    def translate(self, text: str) -> str:
        # Run inference
        model_output = self.model(text)

        # Post-process output to return only the translation text
        translation = model_output[0]["translation_text"]

        return translation


In [12]:
t1_start = perf_counter() 
list(map(lambda x: Translator().translate(x), things))
t1_stop = perf_counter()
print(f"Elapsed time during the {len(things)} requests in seconds:",
                                        t1_stop-t1_start)

KeyboardInterrupt: 